# Overdispersed Poisson Bootstrap Model in Loss Reserving

This notebook is based on CAS Monograph series Number 4 "Using the ODP Bootstrap Model : A Practitioner's Guide" by Mark R. Shapland. This notebook is made because I have an itch to do the coding (or in other words, I can't hacked this problem in Microsoft Excel). Data that being used is the Worker Compensation Data from CAS website.

## Packages

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option("display.precision", 4)

# Importing the Data

In [48]:
data = pd.read_csv('data\wkcomp_pos.csv')
data.head()

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_D,CumPaidLoss_D,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D
0,86,Allstate Ins Co Grp,1988,1988,1,367404,70571,127737,400699,5957,394742,0,281872
1,86,Allstate Ins Co Grp,1988,1989,2,362988,155905,60173,400699,5957,394742,0,281872
2,86,Allstate Ins Co Grp,1988,1990,3,347288,220744,27763,400699,5957,394742,0,281872
3,86,Allstate Ins Co Grp,1988,1991,4,330648,251595,15280,400699,5957,394742,0,281872
4,86,Allstate Ins Co Grp,1988,1992,5,354690,274156,27689,400699,5957,394742,0,281872


In [49]:
data.describe()

,GRCODE,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_D,CumPaidLoss_D,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D
count,13200.0000,13200.0000,13200.0000,13200.0000,13200.0000,13200.0000,13200.0000,13200.0000,13200.0000,13200.0000,13200.0000,1.3200e+04
mean,17153.0530,1992.5000,1997.0000,5.5000,11532.0483,8215.7373,1570.1256,18438.4697,1812.3386,16626.1288,0.7273,3.9714e+04
std,12512.2061,2.8724,4.0622,2.8724,35595.5602,25714.0815,7259.0221,51830.7031,6666.6631,48941.7241,0.4454,1.3013e+05
min,86.0000,1988.0000,1988.0000,1.0000,-59.0000,-338.0000,-4621.0000,-6518.0000,-3522.0000,-9731.0000,0.0000,0.0000e+00
25%,8526.0000,1990.0000,1994.0000,3.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,4.1100e+02
50%,14110.0000,1992.5000,1997.0000,5.5000,544.0000,351.5000,5.0000,1419.0000,144.5000,827.0000,1.0000,2.7320e+03
75%,26983.2500,1995.0000,2000.0000,8.0000,6526.5000,4565.0000,259.2500,11354.2500,1141.0000,9180.5000,1.0000,1.9266e+04
max,44300.0000,1997.0000,2006.0000,10.0000,367404.0000,325322.0000,145296.0000,421223.0000,78730.0000,418755.0000,1.0000,1.0901e+06


In [50]:
data.dtypes

GRCODE                int64
GRNAME               object
AccidentYear          int64
DevelopmentYear       int64
DevelopmentLag        int64
IncurLoss_D           int64
CumPaidLoss_D         int64
BulkLoss_D            int64
EarnedPremDIR_D       int64
EarnedPremCeded_D     int64
EarnedPremNet_D       int64
Single                int64
PostedReserve97_D     int64
dtype: object

In [51]:
print(f'Number of Company {len(data.GRCODE.unique())}')

Number of Company 132



Let just focus on 1 Company with GRCODE 7080

In [52]:
data.GRCODE.unique()

array([   86,   337,   353,   388,   460,   655,   671,   711,   715,
         965,  1066,  1090,  1236,  1252,  1538,  1767,  2135,  2143,
        2623,  2712,  3000,  3034,  3240,  4839,  5010,  5185,  5940,
        6408,  6807,  7080,  7714,  8168,  8427,  8559,  8672,  9466,
       10011, 10022, 10048, 10074, 10191, 10385, 10520, 10561, 10657,
       10659, 10699, 10709, 10781, 10800, 10859, 10874, 11126, 11231,
       11347, 11460, 11703, 12297, 13439, 13501, 13528, 13587, 13641,
       13943, 13994, 14044, 14176, 14257, 14320, 14370, 14508, 14575,
       14974, 15024, 15148, 15199, 15334, 15393, 15792, 15911, 16446,
       18309, 18380, 18538, 18767, 18791, 20451, 21172, 22635, 22900,
       23108, 23140, 23574, 23663, 23876, 24017, 24619, 26433, 26956,
       27065, 27529, 27626, 27905, 27955, 28258, 28886, 30589, 31658,
       31780, 32005, 32875, 33111, 33499, 34576, 35009, 35408, 35904,
       36790, 37370, 38300, 38644, 38687, 38733, 38997, 40126, 41300,
       41394, 41580,

In [61]:
df = data[data['GRCODE'] == 337]
df

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_D,CumPaidLoss_D,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D
100,337,California Cas Grp,1988,1988,1,62679,9558,24619,104437,4658,99779,0,209415
101,337,California Cas Grp,1988,1989,2,64000,22778,13092,104437,4658,99779,0,209415
102,337,California Cas Grp,1988,1990,3,65607,33298,13020,104437,4658,99779,0,209415
103,337,California Cas Grp,1988,1991,4,65292,40348,11540,104437,4658,99779,0,209415
104,337,California Cas Grp,1988,1992,5,62250,45146,9513,104437,4658,99779,0,209415
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,337,California Cas Grp,1997,2002,6,59809,48438,2786,48052,2119,45933,0,209415
196,337,California Cas Grp,1997,2003,7,59230,50775,1358,48052,2119,45933,0,209415
197,337,California Cas Grp,1997,2004,8,59232,52694,1017,48052,2119,45933,0,209415
198,337,California Cas Grp,1997,2005,9,59300,54217,781,48052,2119,45933,0,209415


We are going to show the data in run-off triangle and note, we assume we are in the year of 1997. Meaning that we assume for every data that being collected beyond 1997 is unknown.

In [81]:
df = df[df['DevelopmentYear'] <= 1997]

In [82]:
tr_inc = pd.pivot_table(df, values='IncurLoss_D', index='AccidentYear',
                       columns=['DevelopmentLag'], aggfunc=np.sum)
tr_inc

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,62679.0,64000.0,65607.0,65292.0,62250.0,61185.0,59387.0,57138.0,53473.0,53261.0
1989,62058.0,56660.0,56598.0,56831.0,55294.0,53481.0,51379.0,48446.0,48300.0,NaN
1990,63955.0,61523.0,63166.0,61402.0,59474.0,57613.0,57161.0,56971.0,NaN,NaN
1991,73918.0,75094.0,73188.0,71199.0,71056.0,70831.0,70532.0,NaN,NaN,NaN
1992,77143.0,75133.0,73284.0,73155.0,68284.0,67845.0,NaN,NaN,NaN,NaN
1993,79925.0,78263.0,78163.0,68546.0,67697.0,NaN,NaN,NaN,NaN,NaN
1994,73628.0,73554.0,70736.0,69720.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,69521.0,80281.0,79381.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,69720.0,73181.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
tr_cum = tr_inc.cumsum(axis = 1)
tr_cum

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,62679.0,126679.0,192286.0,257578.0,319828.0,381013.0,440400.0,497538.0,551011.0,604272.0
1989,62058.0,118718.0,175316.0,232147.0,287441.0,340922.0,392301.0,440747.0,489047.0,NaN
1990,63955.0,125478.0,188644.0,250046.0,309520.0,367133.0,424294.0,481265.0,NaN,NaN
1991,73918.0,149012.0,222200.0,293399.0,364455.0,435286.0,505818.0,NaN,NaN,NaN
1992,77143.0,152276.0,225560.0,298715.0,366999.0,434844.0,NaN,NaN,NaN,NaN
1993,79925.0,158188.0,236351.0,304897.0,372594.0,NaN,NaN,NaN,NaN,NaN
1994,73628.0,147182.0,217918.0,287638.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,69521.0,149802.0,229183.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,69720.0,142901.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Basic Chain Ladder

In [86]:
tr_cum.iloc[0,:9]

DevelopmentLag
1     62679.0
2    126679.0
3    192286.0
4    257578.0
5    319828.0
6    381013.0
7    440400.0
8    497538.0
9    551011.0
Name: 1988, dtype: float64

In [89]:
tr_cum.shape

(10, 10)

In [107]:
def dev_fac(triangle = tr_cum, i = 0, shape = tr_cum.shape[1]):
    return np.sum(tr_cum.iloc[:shape - i - 1, i+1])/np.sum(tr_cum.iloc[:shape - i -1, i])

In [112]:
dev_factor = pd.DataFrame(index = np.arange(0,10))
dev_factor.apply(dev_fac,)

Series([], dtype: float64)

In [101]:
shape

10